## The Last code cell has the (currently) final code
## To exit out of the camera feed display, press the 'q' key
## The UI is still ugly, I can't do anything to fix it though. So please fix it. T_T
## Oh, yeah! and the pip installer cell must be run first cuz apparently mediapipe has issues with being locally hosted.

In [1]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os

mp_face_mesh = mp.solutions.face_mesh
DATA_FILE = "medpipe_users.pkl"

def exfaem(landmarks):
    p = np.array([[lm.x, lm.y, lm.z] for lm in landmarks])
    p = p - np.mean(p, axis=0)
    std = np.std(p, axis=0)
    std[std == 0] = 1.0
    p = p / std
    return p.flatten()

def sign_up():
    name = input('Hello! can you tell me your name? ').strip()
    if not name:
        print("I didn't get that, again please? ")
        return
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    print('Capturing... Smile for the camera! (> ▽ <) ')

    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        em = []
        count = 0
        while count < 10:
            ret, frame = cap.read()
            if not ret:
                break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb)
            if results and results.multi_face_landmarks:
                for face_lms in results.multi_face_landmarks:
                    emb = exfaem(face_lms.landmark)
                    em.append(emb)
                    count += 1
                    cv2.putText(frame, f"Captured {count}/10", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            else:
                cv2.putText(frame, "You're not visible", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Sign Up', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

    if em:
        mean_enc = np.mean(em, axis=0)
        if os.path.exists(DATA_FILE):
            with open(DATA_FILE, 'rb') as f:
                users = pickle.load(f)
        else:
            users = {}
        users[name] = mean_enc
        with open(DATA_FILE, 'wb') as f:
            pickle.dump(users, f)
        print(f"User '{name}' registered successfully! ^ ▽ ^)/")
    else:
        print('No valid face captured.')

def sign_in():
    if not os.path.exists(DATA_FILE):
        print("Hmm... looks like you're not on our database. Please sign up first.")
        return
    with open(DATA_FILE, 'rb') as f:
        users = pickle.load(f)
    known_names = list(users.keys())
    known_ems = list(users.values())
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    print("Let's see if we know each other... (press 'q' to quit)")
    recognized = False
    name = None
    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb)
            if results and results.multi_face_landmarks:
                for face_lms in results.multi_face_landmarks:
                    current_emb = exfaem(face_lms.landmark)
                    dists = [np.linalg.norm(current_emb - enc) for enc in known_ems]
                    min_idx = int(np.argmin(dists))
                    if dists[min_idx] < 0.6:
                        name = known_names[min_idx]
                        cv2.putText(frame, f"Welcome {name}!", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                        recognized = True
                        break
                    else:
                        cv2.putText(frame, "Unknown user", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Sign In', frame)
            if recognized or (cv2.waitKey(1) & 0xFF == ord('q')):
                break
    cap.release()
    cv2.destroyAllWindows()
    if recognized:
        print(f"Signed in successfully as {name}.")
        return name
    else:
        print("No matching user found.")
        return None

def main():
    while True:
        print("\n==== FACE LOGIN SYSTEM ====")
        print('1. Sign Up')
        print('2. Sign In')
        print('3. Exit')
        choice = input('Choose an action: ').strip()
        if choice == '1':
            sign_up()
        elif choice == '2':
            sign_in()
        elif choice == '3':
            break
        else:
            print('Invalid. Try again')

if __name__ == '__main__':
    main()

# Sign up sign in code

In [2]:
!pip install mediapipe opencv-python numpy

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os

mp_face_mesh= mp.solutions.face_mesh
DATA_FILE= "medpipe_users.pkl"

def exfaem(landmarks):
    p = np.array([[lm.x, lm.y, lm.z] for lm in landmarks])
    p -= np.mean(p, axis=0)  # Normalize face landmarks: Subtract mean and divide by std to remove scale bias
    p /= np.std(p, axis=0)
    return p.flatten()

def sign_up():
    name = input('Hello! can you tell me your name? '.strip())
    if not name:
        print("I didn't get that, again please? ")
        return
    cap =cv2.VideoCapture(0)
    print('Capturing... Smile for the camera! (> ▽ <) ')
    
    with mp_face_mesh.FaceMesh(
        static_image_mode= False,
        max_num_faces= 1,
        min_detection_confidence= 0.5,
        min_tracking_confidence= 0.5
    ) as face_mesh:
        
        em = []
        count= 0
        while count<10:
            ret, frame= cap.read()
            if not ret:
                break
                
            rgb= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result= face_mesh.process(rgb)
            
            if results.multi_face_landmarks:
                for face_lms in results.multi_face_landmarks:
                    emb= exfaem(face_lms.landmark)
                    em.append(emb)
                    count += 1
                    cv2.putText(frame, f"Captured {count}/10", (50,50),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            else:
                cv2.putText(frame, "You're not visible", (50,50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
            cv2.imshow('Sign Up', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    cap.release()
    cv2.destroyAllWindows()
    
    if em:
        mean_enc= np.mean(em, axis=0)
        
        if os.path.exists(DATA_FILE):
            with open(DATA_FILE, "rb") as f:
                users= pickle.load(f)
        else:
            users= {}
            
        users[name]= mean_enc
        with open(DATA_FILE, "wb") as f:
            pickle.dump(users, f)
            
        print(f" User '{name}' registered successfully! \(^ ▽ ^)/")
    else:
        print('No valid face captured.')
            
def sign_in():
    if not os.path.exists(DATA_FILE):
        print("Hmm... looks like your're not on our database. Please sign up first.")
        return
    
    with open(DATA_FILE, "rb") as f:
        users= pickle.load(f)
        
    known_names= list(users.keys())
    known_ems= list(users.values())
    
    cap= cv2.VideoCapture(0)
    print("Let's see if we know eachother... (press 'q' to quit)")
    
    recognized= False
    with mp_face_mesh.FaceMesh(
        static_image_mode= False,
        max_num_faces= 1,
        min_detection_confidence= 0.5,
        min_tracking_confidence= 0.5
    ) as face_mesh:
        while True:
            ret, frame= cap.read()
            if not ret:
                break
                
            rgb= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result= face_mesh.process(rgb)
            
            if results.multi_face_landmarks:
                for face_lms in results.multi_face_landmarks:
                    current_emb= exfaem(face_lms.landmark)
                    dist= [np.linalg.norm(current_emb - enc) for enc in known_ems]
                    min_idx= np.argmin(dist)
                    if dist[min_idx]<0.6:
                        name= known_names[min_idx]
                        cv2.putText(frame, f"Welcome {name}!", (50,50),
                                   cv2.FONT_KERSHEY_SIMPLEX, 1, (0,255,0), 2)
                        recognized= True
                        break
                    else:
                        cv2.putText(frame, "Unknown user", (50,50),
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                        
            cv2.imshow("Sign In", frame)
            if recognized or (cv2.waitKey(1) & 0xFF == ord('q')):
                break
                
    cap.release()
    cv2.destroyAllWindows()
    
    if recognized:
        print(f"Signed in successfully as {name}.")
        return name
    else:
        print(f"No matching user found.")
        return None
    
def main():
    while True:
        print("\n==== FACE LOGIN SYSTEM ====")
        print('1. Sign Up')
        print('2. Sign In')
        print('3. Exit')
        choice= input('Choose an action: ')
        
        if choice== "1":
            sign_up()
        elif choice== "2":
            sign_in()
        elif choice== "3":
            break
        else:
            print('Invalid. Try again')
            
if _name_=="_main_":
    main()